In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
t1 = torch.zeros(4,2)
print(t1)
print(t1.shape)
print(t1.dtype)
print(t1.device)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
t1_gpu = t1.cuda(device)
print(t1_gpu.device)
print(t1.device)

In [ ]:
""" squeeze & unsqueeze
squeeze함수는 차원이 1인 차원을 제거해준다. 
따로 차원을 설정하지 않으면 1인 차원을 모두 제거한다. 
그리고 차원을 설정해주면 그 차원만 제거한다.

unsqueeze함수는 squeeze함수의 반대로 1인 차원을 생성하는 함수이다. 
그래서 어느 차원에 1인 차원을 생성할 지 꼭 지정해주어야한다.

"""
t2 = torch.rand(2)
print("t2: ")
print(t2)
print(t2.shape)

print("\n---after unsqueeze---")
t2_add_rank = t2.unsqueeze(0)
print(t2_add_rank)
print(t2_add_rank.shape) # 1,2

print(t2_add_rank.unsqueeze(1).shape) # 1, 1 ,2


print(t2_add_rank.squeeze(1).shape) # 1, 2
print(t2_add_rank.squeeze().shape) # 2

In [ ]:
t3 = torch.rand(1,2)
print(t3)
print(t3.shape)

print("\n---after squeeze---")
t3_delete_rank = t3.squeeze(0)
print(t3_delete_rank)
print(t3_delete_rank.shape)


In [ ]:
"""
reshape & permute
"""

t4 = torch.arange(192) # 3 * 8 * 8
print(t4,"\n", t4.shape)

t4 = t4.reshape((3,8,8))
print(f"\n---after reshape---")
print(t4, "\n" ,t4.shape)

t4 = t4.permute(1,2,0)
print("\n---after permute---")
print(t4, "\n", t4.shape)

t4 = t4.permute(2,0,1)
print(t4, "\n" ,t4.shape)

In [ ]:
t4 = torch.rand(12)
print('\n',t4)
t4 = t4.reshape(3,2,2)
print('\n',t4)
t4 = t4.permute(1,2,0) # (2,2,3)
print('\n',t4)

In [ ]:
import numpy as np
import torch

# convert numpy -> tensor
num_array = np.array([[1,2],[3,4]])
t5 = torch.Tensor(num_array)
print(type(num_array))
print(num_array)
print(type(t5))
print(t5)

In [ ]:
t6 = torch.ones_like(t5) # tensor filled w/ value '1' in the same shape of t5
output = t5 + 2 * t6
print(output)

print(t5, "\n",t6)
print("exponential: ")
print(torch.exp(output))

In [ ]:
# dimension reduction
t7 = torch.randn((2,3,4)) # shape: (2,3,4) = (0th dim, 1st dim, 2nd dim)
print(t7)
print()

t7_argmax = torch.argmax(t7, dim=2)
print(t7_argmax.shape)
print(t7_argmax)

print()

print(torch.argmax(t7, 1, keepdims=True))
print(torch.argmax(t7, 1, keepdims=True))
print(torch.argmax(t7, 1, keepdims=True).squeeze(1))

# 데이터셋 & 데이터 로더
## 데이터셋 정의하기

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import random

In [ ]:
class SimpleDataset(Dataset):

    def __init__(self, csv_file=None, root_dir=None):
        self.csv_file = csv_file
        self.root_dir = root_dir

        # set x_data with random length
        n_dataset = 100
        self.x_data = []
        for _ in range(n_dataset):
            random_length = random.randint(5,10)
            self.x_data.append(np.random.randint(0,255,random_length))
        # set y_data
        self.y_data = torch.rand(n_dataset, 1)

        print("SimpleDataset setting done")
        print(f'x_data shape: {len(self.x_data)}')
        print(f'x_data random length (1st, 4th, 46th): {len(self.x_data[0]),len(self.x_data[3]), len(self.x_data[45])}')
        print(f'y_data shape: {self.y_data.shape}')
    
    def __len__(self): 
        return len(self.x_data)

    def __getitem__(self, idx): 
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y
    
    def collate_fn(self, data):
        
        max_len = 10
        batch = []
        for x, y in data:
            x_padded = torch.cat([x, torch.zeros(max_len - x.shape[0])])
            batch.append(x_padded)
        return torch.stack(batch)

dataset = SimpleDataset()

In [ ]:
dataloader = DataLoader(dataset, batch_size=25, collate_fn=dataset.collate_fn, shuffle=True)
# dataloader = DataLoader(dataset, batch_size=25, shuffle=True) # Raise Error!

for batch_idx, samples in enumerate(dataloader):
    print("batch idx: ", batch_idx+1)
    print("sample shape: ", samples.shape)

    if batch_idx==len(dataloader)-1:
        print(samples)

    # ... train continued

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# set dataset
train_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

# set data loader
batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [ ]:
import matplotlib.pyplot as plt

examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)
print("example data shape: ", example_data.shape)

fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Ground Truth: {}".format(example_targets[i]))
  plt.xticks([])
  plt.yticks([])
# fig

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784,128)
        self.fc2 = nn.Linear(128,64)
        self.classifier = nn.Linear(64,10)
    
    def forward(self, x):
        x = x.reshape(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.classifier(x)

network = Net()
print(network)

In [ ]:
def train(model, optim, loss_fn, train_loader, epochs, device):
    for epoch in range(epochs):
        train_loss = 0.0
        model.train()

        for batch in train_loader:
            optim.zero_grad()
            
            input, target = batch
            input = input.to(device)
            target = target.to(device)
            
            output = model(input)
            loss = loss_fn(output, target)
            loss.backward()
            optim.step()
            train_loss += loss.data.item()
        train_loss /= len(train_loader.dataset)

        print(f'Epoch: {epoch+1}, Training Loss: {train_loss}')

def test(model, loss_fn, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    ground_truths = 0
    with torch.no_grad():
        for batch in test_loader:
            input, target = batch
            input = input.to(device)
            target = target.to(device)
            output = model(input)
            loss = loss_fn(output, target)
            test_loss += loss.data.item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
        test_loss /= len(test_loader.dataset)
    
    print(f'Test Loss: {test_loss},\t Accuracy: ({100. * correct / len(test_loader.dataset)}%)\n')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
network.to(device)
print(device)

num_epochs = 3
optimizer = optim.SGD(network.parameters(), lr=1e-2, momentum=0.9)
train(network, optimizer, nn.CrossEntropyLoss(), train_loader, num_epochs, device)
test(network, nn.CrossEntropyLoss(), device)

# Save & Load Model

In [ ]:
# option 1
# save parameters & structure of model

torch.save(network, "./data/first_network")
loaded_network = torch.load("./data/first_network")
print(loaded_network)
print(type(loaded_network))

In [ ]:
# option 2 (recommended)
# maps of each layer's parameters

torch.save(network.state_dict(), "./data/first_network_dict")
state_dict = torch.load("./data/first_network_dict")
print(type(state_dict))

network.add_module("additional_layer", nn.Linear(10,10))
print(*network.modules())

In [ ]:
network.load_state_dict(state_dict, strict=False)
# network.load_state_dict(state_dict, strict=True) # Raise Error!
print(network)

In [ ]:
import torchvision.models as models

resnet18 = models.resnet18(pretrained=True)
print(resnet18)

In [ ]:
num_classes = 10
last_layer_in_features = resnet18.fc.in_features
resnet18.fc = nn.Linear(last_layer_in_features, num_classes)
print(resnet18)